## Music recomendation system

In [ ]:
!pip install pyspark   --upgrade --force-reinstall --no-deps kaggle


  Using cached pyspark-4.0.0-py2.py3-none-any.whl
  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)
Using cached kaggle-1.7.4.5-py3-none-any.whl (181 kB)
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.0
    Uninstalling pyspark-4.0.0:
      Successfully uninstalled pyspark-4.0.0
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.5
    Uninstalling kaggle-1.7.4.5:
      Successfully uninstalled kaggle-1.7.4.5


In [ ]:
!pip install mlflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pyspark.pandas as ps


In [ ]:
!pip install optuna

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"astinseverino","key":"38e1ed063eac09b66211504480c27ac8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s spotify

ref                                                       title                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ---------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
dhruvildave/spotify-charts                                Spotify Charts                      991225045  2022-02-09 19:49:08.213000          20390        333  1.0              
salvatorerastelli/spotify-and-youtube                     Spotify and Youtube                   9386454  2023-03-20 15:43:25.910000          33028        613  1.0              
bricevergnou/spotify-recommendation                       Spotify Recommendation                  30201  2021-07-27 20:06:27.080000          17189        177  1.0              
andrewmvd/spotify-playlists                               Spotify Playlists                   191756954  2021-11-15

In [ ]:
!kaggle datasets download -d   salvatorerastelli/spotify-and-youtube


Dataset URL: https://www.kaggle.com/datasets/salvatorerastelli/spotify-and-youtube
License(s): CC0-1.0
spotify-and-youtube.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip   '/content/spotify-and-youtube.zip'


Archive:  /content/spotify-and-youtube.zip
replace Spotify_Youtube.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Spotify_Youtube.csv     


In [ ]:
df = pd.read_csv('/content/Spotify_Youtube.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        20718 non-null  int64  
 1   Artist            20718 non-null  object 
 2   Url_spotify       20718 non-null  object 
 3   Track             20718 non-null  object 
 4   Album             20718 non-null  object 
 5   Album_type        20718 non-null  object 
 6   Uri               20718 non-null  object 
 7   Danceability      20716 non-null  float64
 8   Energy            20716 non-null  float64
 9   Key               20716 non-null  float64
 10  Loudness          20716 non-null  float64
 11  Speechiness       20716 non-null  float64
 12  Acousticness      20716 non-null  float64
 13  Instrumentalness  20716 non-null  float64
 14  Liveness          20716 non-null  float64
 15  Valence           20716 non-null  float64
 16  Tempo             20716 non-null  float6

In [ ]:
df.drop(['Unnamed: 0','Url_spotify','Uri','Url_youtube','Title','Channel','Views','Likes','Comments','Description','Licensed','official_video','Stream'], axis=1,inplace=True)


In [ ]:
df.head()

,Artist,Track,Album,Album_type,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

,0
Artist,0
Track,0
Album,0
Album_type,0
Danceability,0
Energy,0
Key,0
Loudness,0
Speechiness,0
Acousticness,0


## Modeling

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
df.columns

Index(['Artist', 'Track', 'Album', 'Album_type', 'Danceability', 'Energy',
       'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Duration_ms'],
      dtype='object')

In [ ]:
features = ['Danceability', 'Energy',
       'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Duration_ms']

In [ ]:
# song_features = df[features].copy()

In [ ]:
# Escalado
scaler = StandardScaler()
# X_scaled = scaler.fit_transform(song_features)
X_scaled = scaler.fit_transform(df[features])

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42)
labels = kmeans.fit_predict(X_scaled)

In [ ]:
# Añadimos la etiqueta de cluster al DataFrame
df['cluster'] = labels

In [ ]:
df.head()

,Artist,Track,Album,Album_type,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,cluster
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,0
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,2
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,1
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,1
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,7


In [ ]:
df['cluster'].value_counts()

,count
cluster,
2,5274
1,4561
7,3549
4,3463
5,1782
0,1251
3,824
6,12


## Test 2

In [ ]:
import optuna
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd

# 1) Carga tu DataFrame y define las features
# df = pd.read_csv("tus_datos.csv")  # o como corresponda
features = [
    'Danceability','Energy','Key','Loudness','Speechiness',
    'Acousticness','Instrumentalness','Liveness','Valence','Tempo','Duration_ms'
]
X = df[features].values

# 2) Estandariza
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3) Función objetivo para Optuna
def objective(trial):
    # rango de búsqueda
    n_clusters = trial.suggest_int("n_clusters", 2, 15)
    init_method = trial.suggest_categorical("init", ["k-means++", "random"])
    n_init = trial.suggest_int("n_init", 5, 30)
    max_iter = trial.suggest_int("max_iter", 100, 500)

    km = KMeans(
        n_clusters=n_clusters,
        init=init_method,
        n_init=n_init,
        max_iter=max_iter,
        random_state=42
    )
    labels = km.fit_predict(X_scaled)
    return silhouette_score(X_scaled, labels)

# 4) Crea y corre el estudio
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, timeout=300)

# 5) Imprime resultados y entrena modelo final
print("Mejor silhouette:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

best_km = KMeans(**study.best_params, random_state=42)
df['cluster_optuna'] = best_km.fit_predict(X_scaled)

# 6) Echa un vistazo
print(df[['Artist','Track','cluster_optuna']].head())


[I 2025-07-14 19:32:48,093] A new study created in memory with name: no-name-0f64ee4e-af2f-4f0c-b223-90d69e6a2e4d
[I 2025-07-14 19:32:55,425] Trial 0 finished with value: 0.14408391640685997 and parameters: {'n_clusters': 7, 'init': 'k-means++', 'n_init': 20, 'max_iter': 162}. Best is trial 0 with value: 0.14408391640685997.
[I 2025-07-14 19:33:02,025] Trial 1 finished with value: 0.16995617303046265 and parameters: {'n_clusters': 4, 'init': 'random', 'n_init': 20, 'max_iter': 383}. Best is trial 1 with value: 0.16995617303046265.
[I 2025-07-14 19:33:08,146] Trial 2 finished with value: 0.2791531104942528 and parameters: {'n_clusters': 2, 'init': 'k-means++', 'n_init': 10, 'max_iter': 172}. Best is trial 2 with value: 0.2791531104942528.
[I 2025-07-14 19:33:14,607] Trial 3 finished with value: 0.16995617303046265 and parameters: {'n_clusters': 4, 'init': 'random', 'n_init': 16, 'max_iter': 216}. Best is trial 2 with value: 0.2791531104942528.
[I 2025-07-14 19:33:20,408] Trial 4 finishe

Mejor silhouette: 0.2791531104942528
Mejores hiperparámetros: {'n_clusters': 2, 'init': 'k-means++', 'n_init': 10, 'max_iter': 172}
     Artist                                          Track  cluster_optuna
0  Gorillaz                                 Feel Good Inc.               0
1  Gorillaz                                Rhinestone Eyes               0
2  Gorillaz  New Gold (feat. Tame Impala and Bootie Brown)               0
3  Gorillaz                             On Melancholy Hill               0
4  Gorillaz                                 Clint Eastwood               0


In [ ]:
df[['Artist','Track','cluster_optuna']].head()

,Artist,Track,cluster_optuna
0,Gorillaz,Feel Good Inc.,0
1,Gorillaz,Rhinestone Eyes,0
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),0
3,Gorillaz,On Melancholy Hill,0
4,Gorillaz,Clint Eastwood,0


In [ ]:
df

,Artist,Track,Album,Album_type,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,cluster_optuna
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.7720,138.559,222640.0,0
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.8520,92.761,200173.0,0
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.5510,108.014,215150.0,0
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.5780,120.423,233867.0,0
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.5250,167.953,340920.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20713,SICK LEGEND,JUST DANCE HARDSTYLE,JUST DANCE HARDSTYLE,single,0.582,0.926,5.0,-6.344,0.0328,0.448000,0.000000,0.0839,0.6580,90.002,94667.0,0
20714,SICK LEGEND,SET FIRE TO THE RAIN HARDSTYLE,SET FIRE TO THE RAIN HARDSTYLE,single,0.531,0.936,4.0,-1.786,0.1370,0.028000,0.000000,0.0923,0.6570,174.869,150857.0,0
20715,SICK LEGEND,OUTSIDE HARDSTYLE SPED UP,OUTSIDE HARDSTYLE SPED UP,single,0.443,0.830,4.0,-4.679,0.0647,0.024300,0.000000,0.1540,0.4190,168.388,136842.0,0
20716,SICK LEGEND,ONLY GIRL HARDSTYLE,ONLY GIRL HARDSTYLE,single,0.417,0.767,9.0,-4.004,0.4190,0.356000,0.018400,0.1080,0.5390,155.378,108387.0,0


In [ ]:
## Test 3

In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd

# 1) Carga tu DataFrame y define las features
# df = pd.read_csv("tus_datos.csv")  # o como corresponda
features = [
    'Danceability','Energy','Key','Loudness','Speechiness',
    'Acousticness','Instrumentalness','Liveness','Valence','Tempo','Duration_ms'
]
X = df[features].values

# 2) Estandariza
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configura tu experimento en MLflow
environment_name = "KMeans_Optuna_Experiment"
mlflow.set_experiment(environment_name)

# 3) Función objetivo para Optuna con MLflow tracking
def objective(trial):
    # rango de búsqueda
    n_clusters = trial.suggest_int("n_clusters", 2, 15)
    init_method = trial.suggest_categorical("init", ["k-means++", "random"])
    n_init = trial.suggest_int("n_init", 5, 30)
    max_iter = trial.suggest_int("max_iter", 100, 500)

    # Inicia un run anidado para cada trial
    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        # Log de hiperparámetros
        params = {
            'n_clusters': n_clusters,
            'init': init_method,
            'n_init': n_init,
            'max_iter': max_iter,
            'random_state': 42
        }
        mlflow.log_params(params)

        # Entrena el modelo
        km = KMeans(
            n_clusters=n_clusters,
            init=init_method,
            n_init=n_init,
            max_iter=max_iter,
            random_state=42
        )
        labels = km.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)

        # Log de métrica
        mlflow.log_metric('silhouette_score', score)

    return score

# 4) Crea y corre el estudio Optuna
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, timeout=300)

# 5) Resultados y entrenamiento del modelo final
print("Mejor silhouette:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

# Registra el mejor run encontrado
best_run = study.best_trial.number
print(f"El mejor run fue: trial_{best_run}")

best_km = KMeans(**study.best_params, random_state=42)
best_km.fit(X_scaled)

# Log del modelo final en MLflow
def log_final_model(model, model_name="kmeans_model"):
    mlflow.sklearn.log_model(model, model_name)
    print(f"Modelo final registrado en MLflow como '{model_name}'.")

with mlflow.start_run(run_name="best_model_run"):
    mlflow.log_params(study.best_params)
    mlflow.log_metric('silhouette_score', study.best_value)
    log_final_model(best_km)

# 6) Guarda clusters en el DataFrame

# Asegúrate de usar el mismo escalador si necesitas transformar nuevos datos
df['cluster_optuna'] = best_km.predict(X_scaled)

# 7) Echa un vistazo
print(df[['Artist','Track','cluster_optuna']].head())


2025/07/14 19:47:09 INFO mlflow.tracking.fluent: Experiment with name 'KMeans_Optuna_Experiment' does not exist. Creating a new experiment.
[I 2025-07-14 19:47:09,307] A new study created in memory with name: no-name-60c2e68a-ccbd-412a-a4cd-9395c09be43f
[I 2025-07-14 19:47:24,218] Trial 0 finished with value: 0.14408391640685997 and parameters: {'n_clusters': 7, 'init': 'k-means++', 'n_init': 20, 'max_iter': 162}. Best is trial 0 with value: 0.14408391640685997.
[I 2025-07-14 19:47:33,895] Trial 1 finished with value: 0.16995617303046265 and parameters: {'n_clusters': 4, 'init': 'random', 'n_init': 20, 'max_iter': 383}. Best is trial 1 with value: 0.16995617303046265.
[I 2025-07-14 19:47:40,175] Trial 2 finished with value: 0.2791531104942528 and parameters: {'n_clusters': 2, 'init': 'k-means++', 'n_init': 10, 'max_iter': 172}. Best is trial 2 with value: 0.2791531104942528.
[I 2025-07-14 19:47:46,847] Trial 3 finished with value: 0.16995617303046265 and parameters: {'n_clusters': 4, '

Mejor silhouette: 0.2791531104942528
Mejores hiperparámetros: {'n_clusters': 2, 'init': 'k-means++', 'n_init': 10, 'max_iter': 172}
El mejor run fue: trial_2


2025/07/14 19:52:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 19:52:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo final registrado en MLflow como 'kmeans_model'.
     Artist                                          Track  cluster_optuna
0  Gorillaz                                 Feel Good Inc.               0
1  Gorillaz                                Rhinestone Eyes               0
2  Gorillaz  New Gold (feat. Tame Impala and Bootie Brown)               0
3  Gorillaz                             On Melancholy Hill               0
4  Gorillaz                                 Clint Eastwood               0
